#### Requirement:
* Read dataset
* Pre-processing data
* Apply FPGrowth algorithm to find association rules from this dataset. Find the most popular items in a basket

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, collect_set, count
from pyspark.sql.types import StringType

from pyspark.ml.feature import StringIndexer
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
spark = SparkSession.builder.appName('ex_demo').getOrCreate()

In [ ]:
# Load data
data = spark.read.csv("../../Data/order_products_train.csv", header=True, inferSchema=True)

In [ ]:
data.count()

In [ ]:
data.show()

### Pre-processing data

In [ ]:
data.createOrReplaceTempView("order_products_train")

In [ ]:
products = spark.sql("SELECT DISTINCT product_id FROM order_product_train")
products.count()

In [ ]:
rawData = spark.sql("SELECT * FROM order_product_train")
baskets = rawData.groupBy('order_id').agg(collect_set('product_id').alias('items'))
baskets.createOrReplaceTempView("baskets")

In [ ]:
baskets.show(5, False)

In [ ]:
type(baskets)

In [ ]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.003, minConfidence=0.003)
model = fpGrowth.fit(baskets)

In [ ]:
# Display frequent itemsets
model.freqItemsets.show()

In [ ]:
# Transform examines the input items agaist all the association rules and summary consquents as predict
mostPopularItemInABasket = model.transform(baskets)

In [ ]:
mostPopularItemInABasket.show()

### Use product_name instead of product_id


In [ ]:
product_data = spark.read.csv('../../Data/product.csv', header=True, inferSchema=True)

In [ ]:
product_data.show(5, truncate=False)

In [ ]:
product_data.createOrReplaceTempView("products")

In [ ]:
rawData_1 = spark.sql('''SELECT p.product_name o.order_id 
                         FROM products p 
                         INNER JOIN order_products_train o
                         WHERE o.product_id = p.product_id''')
baskets_1 = rawData.groupBy('order_id').agg(collect_set('product_id').alias('items'))
baskets_1.createOrReplaceTempView("baskets_1")

In [ ]:
baskets_1.head(3)

In [ ]:
fpGrowth_1 = FPGrowth(itemsCol="items", minSupport=0.003, minConfidence=0.003)
model_1 = fpGrowth.fit(baskets_1)

In [ ]:
# Display frequent itemsets
model_1.freqItemsets.show()

In [ ]:
mostPopularItemInABasket_1 = model.transform(baskets_1)

In [ ]:
mostPopularItemInABasket_1.head(3)

In [ ]:
type(mostPopularItemInABasket_1)

In [ ]:
mostPopularItemInABasket_1.printSchema()

In [ ]:
mostPopularItemInABasket_1.createOrReplaceTempView("popular_items")

In [ ]:
DF_cast = mostPopularItemInABasket_1.select('order_id', 
                                            mostPopularItemInABasket_1.items.cast(StringType()), 
                                            mostPopularItemInABasket_1.prediction.cast(StringType()))

In [ ]:
DF_cast.printSchema()

In [ ]:
DF_cast.head(3)